## 药店销量预测
这是[药店销量预测比赛](https://www.kaggle.com/c/rossmann-store-sales)的一个简单尝试参考。<br>

### 数据
<table id="data-files" class="nicetable full roomy align-top border">   
<thead>
    <tr>
        <th colspan="2">File Name</th> 
        <th>Available Formats</th>         
    </tr> 
</thead>

    <tbody  >
        <tr>

            <td class="file-name" colspan="2" rowspan="1">sample_submission.csv</td>
            <td>
<a href="/c/rossmann-store-sales/download/sample_submission.csv.zip" name="sample_submission.csv.zip" onclick="window.Intercom(&#39;trackEvent&#39;,&#39;download_compdata&#39;,{&#39;comp_id&#39;: 4594});">.zip (55.25 kb)</a>                    </td>
        </tr>

    </tbody>
    <tbody  >
        <tr>

            <td class="file-name" colspan="2" rowspan="1">store.csv</td>
            <td>
<a href="/c/rossmann-store-sales/download/store.csv.zip" name="store.csv.zip" onclick="window.Intercom(&#39;trackEvent&#39;,&#39;download_compdata&#39;,{&#39;comp_id&#39;: 4594});">.zip (8.33 kb)</a>                    </td>
        </tr>

    </tbody>
    <tbody  >
        <tr>

            <td class="file-name" colspan="2" rowspan="1">test.csv</td>
            <td>
<a href="/c/rossmann-store-sales/download/test.csv.zip" name="test.csv.zip" onclick="window.Intercom(&#39;trackEvent&#39;,&#39;download_compdata&#39;,{&#39;comp_id&#39;: 4594});">.zip (143.25 kb)</a>                    </td>
        </tr>

    </tbody>
    <tbody  >
        <tr>

            <td class="file-name" colspan="2" rowspan="1">train.csv</td>
            <td>
<a href="/c/rossmann-store-sales/download/train.csv.zip" name="train.csv.zip" onclick="window.Intercom(&#39;trackEvent&#39;,&#39;download_compdata&#39;,{&#39;comp_id&#39;: 4594});">.zip (5.66 mb)</a>                    </td>
        </tr>

    </tbody>
</table>

<p>You are provided with historical sales data for 1,115 Rossmann stores. The task is to forecast the "Sales" column for the test set. Note that some stores in the dataset were temporarily closed for refurbishment.</p>

<h3>Files</h3>
<ul>
<li><strong>train.csv</strong> - historical data including Sales</li>
<li><strong>test.csv</strong> - historical data excluding Sales</li>
<li><strong>sample_submission.csv</strong> - a sample submission file in the correct format</li>
<li><strong>store.csv</strong> - supplemental information about the stores</li>
</ul>
<h3>Data fields</h3>
<p>Most of the fields are self-explanatory. The following are descriptions for those that aren't.</p>
<ul>
<li><strong>Id</strong> - an Id that represents a (Store, Date) duple within the test set</li>
<li><strong>Store</strong> - a unique Id for each store</li>
<li><strong>Sales</strong> - the turnover for any given day (this is what you are predicting)</li>
<li><strong>Customers</strong> - the number of customers on a given day</li>
<li><strong>Open</strong> - an indicator for whether the store was open: 0 = closed, 1 = open</li>
<li><strong>StateHoliday</strong> - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None</li>
<li><strong>SchoolHoliday</strong> - indicates if the (Store, Date) was affected by the closure of public schools</li>
<li><strong>StoreType</strong> - differentiates between 4 different store models: a, b, c, d</li>
<li><strong>Assortment</strong> - describes an assortment level: a = basic, b = extra, c = extended</li>
<li><strong>CompetitionDistance</strong> - distance in meters to the nearest competitor store</li>
<li><strong>CompetitionOpenSince[Month/Year]</strong> - gives the approximate year and month of the time the nearest competitor was opened</li>
<li><strong>Promo</strong> - indicates whether a store is running a promo on that day</li>
<li><strong>Promo2</strong> - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating</li>
<li><strong>Promo2Since[Year/Week]</strong> - describes the year and calendar week when the store started participating in Promo2</li>
<li><strong>PromoInterval</strong> - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store</li>
</ul>

### 引入所需的库

In [1]:
import pandas as pd
import datetime
import csv
import numpy as np
import os
import scipy as sp
import xgboost as xgb
import itertools
import operator
import warnings
warnings.filterwarnings("ignore")

D:\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.base import TransformerMixin
from sklearn import cross_validation
from matplotlib import pylab as plt
plot = True

goal = 'Sales'
myid = 'Id'

### 定义一些变换和评判准则
使用不同的loss function的时候要特别注意这个

In [3]:
def ToWeight(y):
    w = np.zeros(y.shape, dtype=float)
    ind = y != 0
    w[ind] = 1./(y[ind]**2)
    return w

def rmspe(yhat, y):
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean( w * (y - yhat)**2 ))
    return rmspe

def rmspe_xg(yhat, y):
    # y = y.values
    y = y.get_label()
    y = np.exp(y) - 1
    yhat = np.exp(yhat) - 1
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean(w * (y - yhat)**2))
    return "rmspe", rmspe

In [6]:
store = pd.read_csv('store.csv')

In [7]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [8]:
train_df = pd.read_csv('train.csv')

In [9]:
train_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [11]:
test_df = pd.read_csv('test.csv')

In [12]:
test_df.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


### 加载数据

In [14]:
def load_data():
    """
        加载数据，设定数值型和非数值型数据
    """
    store = pd.read_csv('store.csv')
    train_org = pd.read_csv('train.csv',dtype={'StateHoliday':pd.np.string_})
    test_org = pd.read_csv('test.csv',dtype={'StateHoliday':pd.np.string_})
    train = pd.merge(train_org,store, on='Store', how='left')
    test = pd.merge(test_org,store, on='Store', how='left')
    features = test.columns.tolist()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    features_numeric = test.select_dtypes(include=numerics).columns.tolist()
    features_non_numeric = [f for f in features if f not in features_numeric]
    return (train,test,features,features_non_numeric)

### 数据与特征处理

In [15]:
def process_data(train,test,features,features_non_numeric):
    """
        Feature engineering and selection.
    """
    # # FEATURE ENGINEERING
    train = train[train['Sales'] > 0]

    for data in [train,test]:
        # year month day
        data['year'] = data.Date.apply(lambda x: x.split('-')[0])
        data['year'] = data['year'].astype(float)
        data['month'] = data.Date.apply(lambda x: x.split('-')[1])
        data['month'] = data['month'].astype(float)
        data['day'] = data.Date.apply(lambda x: x.split('-')[2])
        data['day'] = data['day'].astype(float)

        # promo interval "Jan,Apr,Jul,Oct"
        data['promojan'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Jan" in x else 0)
        data['promofeb'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Feb" in x else 0)
        data['promomar'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Mar" in x else 0)
        data['promoapr'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Apr" in x else 0)
        data['promomay'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "May" in x else 0)
        data['promojun'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Jun" in x else 0)
        data['promojul'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Jul" in x else 0)
        data['promoaug'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Aug" in x else 0)
        data['promosep'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Sep" in x else 0)
        data['promooct'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Oct" in x else 0)
        data['promonov'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Nov" in x else 0)
        data['promodec'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Dec" in x else 0)

    # # Features set.
    noisy_features = [myid,'Date']
    features = [c for c in features if c not in noisy_features]
    features_non_numeric = [c for c in features_non_numeric if c not in noisy_features]
    features.extend(['year','month','day'])
    # Fill NA
    class DataFrameImputer(TransformerMixin):
        # http://stackoverflow.com/questions/25239958/impute-categorical-missing-values-in-scikit-learn
        def __init__(self):
            """Impute missing values.
            Columns of dtype object are imputed with the most frequent value
            in column.
            Columns of other types are imputed with mean of column.
            """
        def fit(self, X, y=None):
            self.fill = pd.Series([X[c].value_counts().index[0] # mode
                if X[c].dtype == np.dtype('O') else X[c].mean() for c in X], # mean
                index=X.columns)
            return self
        def transform(self, X, y=None):
            return X.fillna(self.fill)
    train = DataFrameImputer().fit_transform(train)
    test = DataFrameImputer().fit_transform(test)
    # Pre-processing non-numberic values
    le = LabelEncoder()
    for col in features_non_numeric:
        le.fit(list(train[col])+list(test[col]))
        train[col] = le.transform(train[col])
        test[col] = le.transform(test[col])
    # LR和神经网络这种模型都对输入数据的幅度极度敏感，请先做归一化操作
    scaler = StandardScaler()
    for col in set(features) - set(features_non_numeric) - \
      set([]): # TODO: add what not to scale
        scaler.fit(list(train[col])+list(test[col]))
        train[col] = scaler.transform(train[col])
        test[col] = scaler.transform(test[col])
    return (train,test,features,features_non_numeric)

### 训练与分析

In [16]:
def XGB_native(train,test,features,features_non_numeric):
    depth = 13
    eta = 0.01
    ntrees = 8000
    mcw = 3
    params = {"objective": "reg:linear",
              "booster": "gbtree",
              "eta": eta,
              "max_depth": depth,
              "min_child_weight": mcw,
              "subsample": 0.9,
              "colsample_bytree": 0.7,
              "silent": 1
              }
    print "Running with params: " + str(params)
    print "Running with ntrees: " + str(ntrees)
    print "Running with features: " + str(features)

    # Train model with local split
    tsize = 0.05
    X_train, X_test = cross_validation.train_test_split(train, test_size=tsize)
    dtrain = xgb.DMatrix(X_train[features], np.log(X_train[goal] + 1))
    dvalid = xgb.DMatrix(X_test[features], np.log(X_test[goal] + 1))
    watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    gbm = xgb.train(params, dtrain, ntrees, evals=watchlist, early_stopping_rounds=100, feval=rmspe_xg, verbose_eval=True)
    train_probs = gbm.predict(xgb.DMatrix(X_test[features]))
    indices = train_probs < 0
    train_probs[indices] = 0
    error = rmspe(np.exp(train_probs) - 1, X_test[goal].values)
    print error

    # Predict and Export
    test_probs = gbm.predict(xgb.DMatrix(test[features]))
    indices = test_probs < 0
    test_probs[indices] = 0
    submission = pd.DataFrame({myid: test[myid], goal: np.exp(test_probs) - 1})
    if not os.path.exists('result/'):
        os.makedirs('result/')
    submission.to_csv("./result/dat-xgb_d%s_eta%s_ntree%s_mcw%s_tsize%s.csv" % (str(depth),str(eta),str(ntrees),str(mcw),str(tsize)) , index=False)
    # Feature importance
    if plot:
      outfile = open('xgb.fmap', 'w')
      i = 0
      for feat in features:
          outfile.write('{0}\t{1}\tq\n'.format(i, feat))
          i = i + 1
      outfile.close()
      importance = gbm.get_fscore(fmap='xgb.fmap')
      importance = sorted(importance.items(), key=operator.itemgetter(1))
      df = pd.DataFrame(importance, columns=['feature', 'fscore'])
      df['fscore'] = df['fscore'] / df['fscore'].sum()
      # Plotitup
      plt.figure()
      df.plot()
      df.plot(kind='barh', x='feature', y='fscore', legend=False, figsize=(25, 15))
      plt.title('XGBoost Feature Importance')
      plt.xlabel('relative importance')
      plt.gcf().savefig('Feature_Importance_xgb_d%s_eta%s_ntree%s_mcw%s_tsize%s.png' % (str(depth),str(eta),str(ntrees),str(mcw),str(tsize)))

In [ ]:
print "=> 载入数据中..."
train,test,features,features_non_numeric = load_data()
print "=> 处理数据与特征工程..."
train,test,features,features_non_numeric = process_data(train,test,features,features_non_numeric)
print "=> 使用XGBoost建模..."
XGB_native(train,test,features,features_non_numeric)

=> 载入数据中...
=> 处理数据与特征工程...
=> 使用XGBoost建模...
Running with params: {'subsample': 0.9, 'eta': 0.01, 'colsample_bytree': 0.7, 'silent': 1, 'objective': 'reg:linear', 'max_depth': 13, 'min_child_weight': 3, 'booster': 'gbtree'}
Running with ntrees: 8000
Running with features: ['Store', 'DayOfWeek', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'year', 'month', 'day']
Will train until train error hasn't decreased in 100 rounds.
[0]	eval-rmspe:0.999864	train-rmspe:0.999864
[1]	eval-rmspe:0.999838	train-rmspe:0.999838
[2]	eval-rmspe:0.999810	train-rmspe:0.999810
[3]	eval-rmspe:0.999780	train-rmspe:0.999780
[4]	eval-rmspe:0.999748	train-rmspe:0.999747
[5]	eval-rmspe:0.999714	train-rmspe:0.999713
[6]	eval-rmspe:0.999677	train-rmspe:0.999676
[7]	eval-rmspe:0.999638	train-rmspe:0.999637
[8]	eval-rmspe:0.999596	train-rmspe:

[164]	eval-rmspe:0.788333	train-rmspe:0.788352
[165]	eval-rmspe:0.785042	train-rmspe:0.785074
[166]	eval-rmspe:0.781736	train-rmspe:0.781783
[167]	eval-rmspe:0.778404	train-rmspe:0.778463
[168]	eval-rmspe:0.775061	train-rmspe:0.775135
[169]	eval-rmspe:0.771702	train-rmspe:0.771789
[170]	eval-rmspe:0.768329	train-rmspe:0.768432
[171]	eval-rmspe:0.764938	train-rmspe:0.765057
[172]	eval-rmspe:0.761536	train-rmspe:0.761674
[173]	eval-rmspe:0.758127	train-rmspe:0.758283
[174]	eval-rmspe:0.754699	train-rmspe:0.754874
[175]	eval-rmspe:0.751271	train-rmspe:0.751466
[176]	eval-rmspe:0.747823	train-rmspe:0.748040
[177]	eval-rmspe:0.744358	train-rmspe:0.744597
[178]	eval-rmspe:0.740882	train-rmspe:0.741144
[179]	eval-rmspe:0.737399	train-rmspe:0.737682
[180]	eval-rmspe:0.733908	train-rmspe:0.734213
[181]	eval-rmspe:0.730405	train-rmspe:0.730733
[182]	eval-rmspe:0.726894	train-rmspe:0.727244
[183]	eval-rmspe:0.723370	train-rmspe:0.723744
[184]	eval-rmspe:0.719844	train-rmspe:0.720248
[185]	eval-rm

[339]	eval-rmspe:0.276859	train-rmspe:0.288829
[340]	eval-rmspe:0.275332	train-rmspe:0.287344
[341]	eval-rmspe:0.273809	train-rmspe:0.285822
[342]	eval-rmspe:0.272318	train-rmspe:0.284467
[343]	eval-rmspe:0.270847	train-rmspe:0.283008
[344]	eval-rmspe:0.269432	train-rmspe:0.281597
[345]	eval-rmspe:0.268001	train-rmspe:0.280151
[346]	eval-rmspe:0.266466	train-rmspe:0.278762
[347]	eval-rmspe:0.264954	train-rmspe:0.277394
[348]	eval-rmspe:0.263548	train-rmspe:0.276107
[349]	eval-rmspe:0.262195	train-rmspe:0.274878
[350]	eval-rmspe:0.260834	train-rmspe:0.273657
[351]	eval-rmspe:0.259524	train-rmspe:0.272310
[352]	eval-rmspe:0.258195	train-rmspe:0.271119
[353]	eval-rmspe:0.256742	train-rmspe:0.269816
[354]	eval-rmspe:0.255489	train-rmspe:0.268654
[355]	eval-rmspe:0.254067	train-rmspe:0.267360
[356]	eval-rmspe:0.252844	train-rmspe:0.266126
[357]	eval-rmspe:0.251598	train-rmspe:0.264860
[358]	eval-rmspe:0.250414	train-rmspe:0.263786
[359]	eval-rmspe:0.249259	train-rmspe:0.262748
[360]	eval-rm

[514]	eval-rmspe:0.171519	train-rmspe:0.191216
[515]	eval-rmspe:0.171290	train-rmspe:0.191012
[516]	eval-rmspe:0.171116	train-rmspe:0.190628
[517]	eval-rmspe:0.170919	train-rmspe:0.190497
[518]	eval-rmspe:0.170855	train-rmspe:0.190444
[519]	eval-rmspe:0.170770	train-rmspe:0.190391
[520]	eval-rmspe:0.170653	train-rmspe:0.190111
[521]	eval-rmspe:0.170475	train-rmspe:0.189990
[522]	eval-rmspe:0.170263	train-rmspe:0.189803
[523]	eval-rmspe:0.170064	train-rmspe:0.189620
[524]	eval-rmspe:0.169935	train-rmspe:0.189441
[525]	eval-rmspe:0.169878	train-rmspe:0.189412
[526]	eval-rmspe:0.169804	train-rmspe:0.189209
[527]	eval-rmspe:0.169720	train-rmspe:0.189134
[528]	eval-rmspe:0.169608	train-rmspe:0.188914
[529]	eval-rmspe:0.169363	train-rmspe:0.188726
[530]	eval-rmspe:0.169152	train-rmspe:0.188514
[531]	eval-rmspe:0.168967	train-rmspe:0.188373
[532]	eval-rmspe:0.168739	train-rmspe:0.188167
[533]	eval-rmspe:0.168661	train-rmspe:0.188114
[534]	eval-rmspe:0.168551	train-rmspe:0.188012
[535]	eval-rm

[689]	eval-rmspe:0.150245	train-rmspe:0.168585
[690]	eval-rmspe:0.150225	train-rmspe:0.168524
[691]	eval-rmspe:0.150038	train-rmspe:0.168341
[692]	eval-rmspe:0.149799	train-rmspe:0.168129
[693]	eval-rmspe:0.149730	train-rmspe:0.167991
[694]	eval-rmspe:0.149673	train-rmspe:0.167937
[695]	eval-rmspe:0.149581	train-rmspe:0.167869
[696]	eval-rmspe:0.149553	train-rmspe:0.167859
[697]	eval-rmspe:0.149488	train-rmspe:0.167590
[698]	eval-rmspe:0.149389	train-rmspe:0.167510
[699]	eval-rmspe:0.149327	train-rmspe:0.167244
[700]	eval-rmspe:0.149280	train-rmspe:0.167018
[701]	eval-rmspe:0.149055	train-rmspe:0.166845
[702]	eval-rmspe:0.148881	train-rmspe:0.166711
[703]	eval-rmspe:0.148716	train-rmspe:0.166517
[704]	eval-rmspe:0.148622	train-rmspe:0.166438
[705]	eval-rmspe:0.148409	train-rmspe:0.166161
[706]	eval-rmspe:0.148268	train-rmspe:0.166040
[707]	eval-rmspe:0.148143	train-rmspe:0.165923
[708]	eval-rmspe:0.147968	train-rmspe:0.165774
[709]	eval-rmspe:0.147951	train-rmspe:0.165760
[710]	eval-rm

...

[7194]	eval-rmspe:0.096361	train-rmspe:0.051715
[7195]	eval-rmspe:0.096361	train-rmspe:0.051712
[7196]	eval-rmspe:0.096362	train-rmspe:0.051709
[7197]	eval-rmspe:0.096362	train-rmspe:0.051707
[7198]	eval-rmspe:0.096361	train-rmspe:0.051702
[7199]	eval-rmspe:0.096361	train-rmspe:0.051698
[7200]	eval-rmspe:0.096356	train-rmspe:0.051690
[7201]	eval-rmspe:0.096356	train-rmspe:0.051688
[7202]	eval-rmspe:0.096356	train-rmspe:0.051686
[7203]	eval-rmspe:0.096356	train-rmspe:0.051684
[7204]	eval-rmspe:0.096356	train-rmspe:0.051683
[7205]	eval-rmspe:0.096358	train-rmspe:0.051679
[7206]	eval-rmspe:0.096358	train-rmspe:0.051672
[7207]	eval-rmspe:0.096359	train-rmspe:0.051667
[7208]	eval-rmspe:0.096356	train-rmspe:0.051662
[7209]	eval-rmspe:0.096358	train-rmspe:0.051656
[7210]	eval-rmspe:0.096357	train-rmspe:0.051656
[7211]	eval-rmspe:0.096358	train-rmspe:0.051653
[7212]	eval-rmspe:0.096359	train-rmspe:0.051649
[7213]	eval-rmspe:0.096359	train-rmspe:0.051647
[7214]	eval-rmspe:0.096360	train-rmspe:0

[7365]	eval-rmspe:0.096369	train-rmspe:0.051079
[7366]	eval-rmspe:0.096369	train-rmspe:0.051069
[7367]	eval-rmspe:0.096367	train-rmspe:0.051066
[7368]	eval-rmspe:0.096367	train-rmspe:0.051060
[7369]	eval-rmspe:0.096367	train-rmspe:0.051053
[7370]	eval-rmspe:0.096367	train-rmspe:0.051050
[7371]	eval-rmspe:0.096368	train-rmspe:0.051045
[7372]	eval-rmspe:0.096368	train-rmspe:0.051045
[7373]	eval-rmspe:0.096369	train-rmspe:0.051044
[7374]	eval-rmspe:0.096369	train-rmspe:0.051043
[7375]	eval-rmspe:0.096370	train-rmspe:0.051041
[7376]	eval-rmspe:0.096372	train-rmspe:0.051038
[7377]	eval-rmspe:0.096371	train-rmspe:0.051033
[7378]	eval-rmspe:0.096370	train-rmspe:0.051027
[7379]	eval-rmspe:0.096370	train-rmspe:0.051025
[7380]	eval-rmspe:0.096370	train-rmspe:0.051024
[7381]	eval-rmspe:0.096371	train-rmspe:0.051021
[7382]	eval-rmspe:0.096372	train-rmspe:0.051017
[7383]	eval-rmspe:0.096373	train-rmspe:0.051013
[7384]	eval-rmspe:0.096374	train-rmspe:0.051008
[7385]	eval-rmspe:0.096373	train-rmspe:0

[7536]	eval-rmspe:0.096404	train-rmspe:0.050472
[7537]	eval-rmspe:0.096404	train-rmspe:0.050470
[7538]	eval-rmspe:0.096403	train-rmspe:0.050467
[7539]	eval-rmspe:0.096404	train-rmspe:0.050464
[7540]	eval-rmspe:0.096405	train-rmspe:0.050462
[7541]	eval-rmspe:0.096405	train-rmspe:0.050456
[7542]	eval-rmspe:0.096400	train-rmspe:0.050450
[7543]	eval-rmspe:0.096400	train-rmspe:0.050447
[7544]	eval-rmspe:0.096400	train-rmspe:0.050440
[7545]	eval-rmspe:0.096400	train-rmspe:0.050437
[7546]	eval-rmspe:0.096400	train-rmspe:0.050433
[7547]	eval-rmspe:0.096400	train-rmspe:0.050430
[7548]	eval-rmspe:0.096400	train-rmspe:0.050426
[7549]	eval-rmspe:0.096400	train-rmspe:0.050426
[7550]	eval-rmspe:0.096400	train-rmspe:0.050424
[7551]	eval-rmspe:0.096400	train-rmspe:0.050418
[7552]	eval-rmspe:0.096400	train-rmspe:0.050415
[7553]	eval-rmspe:0.096400	train-rmspe:0.050411
[7554]	eval-rmspe:0.096401	train-rmspe:0.050409
[7555]	eval-rmspe:0.096401	train-rmspe:0.050405
[7556]	eval-rmspe:0.096396	train-rmspe:0

[7707]	eval-rmspe:0.096408	train-rmspe:0.049832
[7708]	eval-rmspe:0.096408	train-rmspe:0.049828
[7709]	eval-rmspe:0.096408	train-rmspe:0.049826
[7710]	eval-rmspe:0.096408	train-rmspe:0.049820
[7711]	eval-rmspe:0.096408	train-rmspe:0.049817
[7712]	eval-rmspe:0.096409	train-rmspe:0.049816
[7713]	eval-rmspe:0.096409	train-rmspe:0.049812
[7714]	eval-rmspe:0.096410	train-rmspe:0.049809
[7715]	eval-rmspe:0.096411	train-rmspe:0.049806
[7716]	eval-rmspe:0.096410	train-rmspe:0.049804
[7717]	eval-rmspe:0.096411	train-rmspe:0.049800
[7718]	eval-rmspe:0.096411	train-rmspe:0.049800
[7719]	eval-rmspe:0.096410	train-rmspe:0.049795
[7720]	eval-rmspe:0.096411	train-rmspe:0.049791
[7721]	eval-rmspe:0.096411	train-rmspe:0.049789
[7722]	eval-rmspe:0.096410	train-rmspe:0.049784
[7723]	eval-rmspe:0.096409	train-rmspe:0.049779
[7724]	eval-rmspe:0.096409	train-rmspe:0.049779
[7725]	eval-rmspe:0.096408	train-rmspe:0.049774
[7726]	eval-rmspe:0.096408	train-rmspe:0.049770
[7727]	eval-rmspe:0.096407	train-rmspe:0

[7878]	eval-rmspe:0.096422	train-rmspe:0.049253
[7879]	eval-rmspe:0.096422	train-rmspe:0.049247
[7880]	eval-rmspe:0.096422	train-rmspe:0.049245
[7881]	eval-rmspe:0.096422	train-rmspe:0.049244
[7882]	eval-rmspe:0.096422	train-rmspe:0.049242
[7883]	eval-rmspe:0.096422	train-rmspe:0.049239
[7884]	eval-rmspe:0.096421	train-rmspe:0.049236
[7885]	eval-rmspe:0.096422	train-rmspe:0.049231
[7886]	eval-rmspe:0.096422	train-rmspe:0.049228
[7887]	eval-rmspe:0.096424	train-rmspe:0.049221
[7888]	eval-rmspe:0.096424	train-rmspe:0.049217
[7889]	eval-rmspe:0.096424	train-rmspe:0.049216
[7890]	eval-rmspe:0.096426	train-rmspe:0.049210
[7891]	eval-rmspe:0.096425	train-rmspe:0.049205
[7892]	eval-rmspe:0.096425	train-rmspe:0.049200
[7893]	eval-rmspe:0.096426	train-rmspe:0.049199
[7894]	eval-rmspe:0.096424	train-rmspe:0.049194
[7895]	eval-rmspe:0.096420	train-rmspe:0.049190
[7896]	eval-rmspe:0.096422	train-rmspe:0.049186
[7897]	eval-rmspe:0.096422	train-rmspe:0.049186
[7898]	eval-rmspe:0.096422	train-rmspe:0